##### Copyright 2019 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2019 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS, 
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Классификация статей Bangla с TF-Hub

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/hub/tutorials/bangla_article_classifier"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> Посмотреть на TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/bangla_article_classifier.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Запустить в Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/hub/blob/master/examples/colab/bangla_article_classifier.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Посмотреть источник на GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/hub/examples/colab/bangla_article_classifier.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> Скачать блокнот</a></td>
</table>

Внимание: В дополнение к установке пакетов Python с помощью pip, этот ноутбук использует `sudo apt install` для установки системных пакетов: `unzip` .

Этот колаб демонстрирует использование [Tensorflow Hub](https://www.tensorflow.org/hub/) для классификации текста на неанглийских / местных языках. Здесь мы выбираем [Bangla](https://en.wikipedia.org/wiki/Bengali_language) в качестве местного языка и используем предварительно подготовленные вложения слов для решения задачи классификации с несколькими классами, где мы классифицируем новостные статьи Bangla по 5 категориям. Предварительно обученные вложения для Bangla происходят из [fastText](https://fasttext.cc/docs/en/crawl-vectors.html) , библиотеки Facebook, которая выпустила предварительно обученные векторы слов для 157 языков.

Мы будем использовать экспортер встроенных встроенных функций TF-Hub для преобразования вложений слов в модуль вставки текста, а затем использовать модуль для обучения классификатора с [помощью](https://www.tensorflow.org/api_docs/python/tf/keras) tf.keras, высокоуровневого пользовательского API Tensorflow для построения моделей глубокого обучения. Даже если мы используем здесь встраивания fastText, можно экспортировать любые другие вложения, предварительно подготовленные для других задач, и быстро получить результаты с помощью концентратора Tensorflow. 

## Настроить

In [0]:
%%bash
# https://github.com/pypa/setuptools/issues/1694#issuecomment-466010982
pip install gdown --no-use-pep517

In [0]:
%%bash
sudo apt-get install -y unzip

In [0]:
import os

import tensorflow as tf
import tensorflow_hub as hub

import gdown
import numpy as np
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Dataset

Мы будем использовать [BARD](https://www.researchgate.net/publication/328214545_BARD_Bangla_Article_Classification_Using_a_New_Comprehensive_Dataset) (Bangla Article Dataset), который содержит около 3,76,226 статей, собранных с разных новостных порталов Bangla и помеченных 5 категориями: экономика, государство, международный, спорт и развлечения. Мы загружаем файл с Google Drive по этой ( [bit.ly/BARD_DATASET](bit.ly/BARD_DATASET) ) ссылке, ссылающейся на [этот](https://github.com/tanvirfahim15/BARD-Bangla-Article-Classifier) репозиторий GitHub.


In [0]:
gdown.download(
    url='https://drive.google.com/uc?id=1Ag0jd21oRwJhVFIBohmX_ogeojVtapLy',
    output='bard.zip',
    quiet=True
)

In [0]:
%%bash
unzip -qo bard.zip

# Экспорт предварительно обученных векторов слов в модуль TF-Hub

TF-концентратор обеспечивает некоторые полезные скрипты для преобразования вложения слов в TF-концентраторов модулей текст встраиванию [здесь](https://github.com/tensorflow/hub/tree/master/examples/text_embeddings_v2) . Чтобы создать модуль для Bangla или любых других языков, нам просто нужно загрузить слово, встраивающее файл .txt или .vec в тот же каталог, что и export_v2.py, и запустить скрипт.

Экспортер считывает векторы внедрения и экспортирует их в Tensorflow [SavedModel](https://www.tensorflow.org/beta/guide/saved_model) . SavedModel содержит полную программу TensorFlow, включая веса и график. TF-Hub может загрузить SavedModel как [модуль,](https://www.tensorflow.org/hub/api_docs/python/hub/Module) который мы будем использовать для построения модели для классификации текста. Поскольку мы используем tf.keras для построения модели, мы будем использовать [hub.KerasLayer,](https://www.tensorflow.org/hub/api_docs/python/hub/KerasLayer) который предоставляет оболочку для модуля-концентратора, который будет использоваться в качестве слоя Keras.

Во- первых , мы получим наши слова вложения от FastText и внедренный экспортер из TF-концентратора [репо](https://github.com/tensorflow/hub) .


In [0]:
%%bash
curl -O https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bn.300.vec.gz
curl -O https://raw.githubusercontent.com/tensorflow/hub/master/examples/text_embeddings_v2/export_v2.py
gunzip -qf cc.bn.300.vec.gz --k

Затем мы запустим скрипт экспорта для нашего файла встраивания. Поскольку встраивание fastText имеет строку заголовка и довольно велико (около 3,3 ГБ для bangla после преобразования в модуль), мы игнорируем первую строку и экспортируем только первые 100 000 токенов в модуль встраивания текста.

In [0]:
%%bash
python export_v2.py --embedding_file=cc.bn.300.vec --export_path=text_module --num_lines_to_ignore=1 --num_lines_to_use=100000

In [0]:
module_path = "text_module"
embedding_layer = hub.KerasLayer(module_path, trainable=False)

The text embedding module takes a batch of sentences in a 1D tensor of strings as input and outputs the embedding vectors of shape (batch_size, embedding_dim) corresponding to the sentences. It preprocesses the input by splitting on spaces. Word embeddings are combined to sentence embeddings with the `sqrtn` combiner(See [here](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup_sparse)). For demonstration we pass a list of Bangla words as input and get the corresponding embedding vectors.

In [0]:
embedding_layer(['বাস', 'বসবাস', 'ট্রেন', 'যাত্রী', 'ট্রাক']) 

# Преобразовать в набор данных Tensorflow


Поскольку набор данных действительно большой, вместо загрузки всего набора данных в память, мы будем использовать генератор для получения выборок во время выполнения в пакетах с использованием [функций набора данных Tensorflow](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) . Набор данных также очень несбалансирован, поэтому перед использованием генератора мы перетасуем набор данных.


In [0]:
dir_names = ['economy', 'sports', 'entertainment', 'state', 'international']

file_paths = []
labels = []
for i, dir in enumerate(dir_names):
  file_names = ["/".join([dir, name]) for name in os.listdir(dir)]
  file_paths += file_names
  labels += [i] * len(os.listdir(dir))
  
np.random.seed(42)
permutation = np.random.permutation(len(file_paths))

file_paths = np.array(file_paths)[permutation]
labels = np.array(labels)[permutation]

Мы можем проверить распределение меток в примерах обучения и проверки после перетасовки.

In [0]:
train_frac = 0.8
train_size = int(len(file_paths) * train_frac)

In [0]:
# plot training vs validation distribution
plt.subplot(1, 2, 1)
plt.hist(labels[0:train_size])
plt.title("Train labels")
plt.subplot(1, 2, 2)
plt.hist(labels[train_size:])
plt.title("Validation labels")
plt.tight_layout()

Чтобы создать [набор данных](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) с использованием генератора, мы сначала пишем функцию генератора, которая читает каждую из статей из file_paths и метки из массива меток и выдает один обучающий пример на каждом шаге. Мы передаем эту функцию генератора методу [tf.data.Dataset.from_generator](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_generator) и указываем типы вывода. Каждый обучающий пример представляет собой кортеж, содержащий статью с типом данных tf.string и метку с горячим кодированием. Мы разбили набор данных с разбивкой валидации поезда на 80-20, используя метод [`skip`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#skip) и [`take`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#take) .

In [0]:
def load_file(path, label):
    return tf.io.read_file(path), label

In [0]:
def make_datasets(train_size):
  batch_size = 256

  train_files = file_paths[:train_size]
  train_labels = labels[:train_size]
  train_ds = tf.data.Dataset.from_tensor_slices((train_files, train_labels))
  train_ds = train_ds.map(load_file).shuffle(5000)
  train_ds = train_ds.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

  test_files = file_paths[train_size:]
  test_labels = labels[train_size:]
  test_ds = tf.data.Dataset.from_tensor_slices((test_files, test_labels))
  test_ds = test_ds.map(load_file)
  test_ds = test_ds.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)


  return train_ds, test_ds

In [0]:
train_data, validation_data = make_datasets(train_size)

# Модельное обучение и оценка

Поскольку мы уже добавили обертку вокруг нашего модуля, чтобы использовать его как любой другой слой в керасе, мы можем создать небольшую [последовательную](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) модель, представляющую собой линейный стек слоев. Мы можем добавить наш модуль встраивания текста с `model.add` как и любой другой слой. Мы компилируем модель, определяя потери и оптимизатор, и обучаем ее в течение 10 эпох. API `tf.keras` может обрабатывать наборы данных tenorflow в качестве входных данных, поэтому мы можем передать экземпляр набора данных методу fit для обучения модели. Поскольку мы используем функцию генератора, tf.data будет обрабатывать генерацию сэмплов, их пакетирование и подачу в модель.

## Модель

In [0]:
def create_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=[], dtype=tf.string),
    embedding_layer,
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(5),
  ])
  model.compile(loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
      optimizer="adam", metrics=['accuracy'])
  return model

In [0]:
model = create_model()
# Create earlystopping callback
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3)

## Повышение квалификации

In [0]:
history = model.fit(train_data, 
                    validation_data=validation_data, 
                    epochs=5, 
                    callbacks=[early_stopping_callback])

## оценка

Мы можем визуализировать кривые точности и потерь для данных обучения и проверки, используя объект `history` возвращаемый методом `fit` который содержит значение потерь и точности для каждой эпохи.

In [0]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## прогнозирование

Мы можем получить прогнозы для данных проверки и проверить матрицу путаницы, чтобы увидеть производительность модели для каждого из 5 классов. Как `predict` метод возвращает нас к - й массив для вероятностей для каждого класса , который мы преобразуем к классу меток с использованием `np.argmax` .

In [0]:
y_pred = model.predict(validation_data)

In [0]:
y_pred = np.argmax(y_pred, axis=1)

In [0]:
samples = file_paths[0:3]
for i, sample in enumerate(samples):
  f = open(sample)
  text = f.read()
  print(text[0:100])
  print("True Class: ", sample.split("/")[0])
  print("Predicted Class: ", dir_names[y_pred[i]])
  f.close()
  

## Сравнить производительность

Теперь мы можем взять правильные метки для данных проверки из `labels` и сравнить их с нашими прогнозами, чтобы получить [классификационный_отчет](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) . 

In [0]:
y_true = np.array(labels[train_size:])

In [0]:
print(classification_report(y_true, y_pred, target_names=dir_names))

Мы также можем сравнить производительность нашей модели с опубликованными результатами, полученными в оригинальной [статье, в](https://www.researchgate.net/publication/328214545_BARD_Bangla_Article_Classification_Using_a_New_Comprehensive_Dataset) которой сообщается о точности 0,96. Авторы описали множество шагов предварительной обработки, выполненных с набором данных, таких как удаление знаков препинания и цифр, удаление 25 самых популярных стоп-слов. Как мы можем видеть из классификационного отчета, мы также получаем точность и точность 0,96 после обучения всего 5 эпох без какой-либо предварительной обработки!

В этом примере, когда мы создали слой Keras из нашего модуля внедрения, мы установили `trainable=False` , что означает, что веса внедрения не будут обновляться во время обучения. Попробуйте установить его в True, чтобы достичь точности 97% с этим набором данных только с 2 эпохами. 